In [6]:
import pandas as pd

from IPython.display import display
from geopy.geocoders import Nominatim

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [7]:
data = pd.read_csv( '../Datasets/data-filter.csv' )

In [8]:
# new date format
data['year'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y' )
data['year_week'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y-%U' )


In [9]:
# create is_waterfront row
data['is_waterfront'] = data['waterfront'].apply( lambda x: 'no' if x == 0 else 'yes' )

In [22]:
# create level for each quartile of price
quartile = list( data['price'].quantile([0.25, 0.5, 0.75]) )

data['price_level'] = data['price'].apply( lambda x: 'level_one'   if x <= quartile[0] else
                                                     'level_two'   if ( x > quartile[0] ) & ( x <= quartile[1] ) else
                                                     'level_three' if ( x > quartile[1] ) & ( x <= quartile[2] ) else
                                                     'level_four' )

[321837.5, 450000.0, 645000.0]

In [25]:
data[['price', 'price_level']].sample(20)

,price,price_level
17889,440000.00,level_two
17368,421200.00,level_two
4994,637000.00,level_three
16324,180000.00,level_one
12534,530000.00,level_three
1375,320000.00,level_one
15993,465000.00,level_three
6770,1335000.00,level_four
14080,260000.00,level_one
6064,228000.00,level_one


In [69]:
# create a condition status
condition = list( data['condition'].unique() )
condition.sort()

data['status'] = data['condition'].apply( lambda x: 'bad' if x <= condition[1] else
                                                    'regular' if x == condition[2] else
                                                    'good' if x == condition[3] else 'excelent' )

In [80]:
data.loc[data['condition'] == 5, ['condition','status']].sample(10)

,condition,status
8253,5,excelent
721,5,excelent
11720,5,excelent
8108,5,excelent
16118,5,excelent
10486,5,excelent
1336,5,excelent
8892,5,excelent
18512,5,excelent
18547,5,excelent


In [ ]:
# create adress data

# create empty rows:
data['road'] = 'NA'
data['house_number'] = 'NA'
data['neighbourhood'] = 'NA'
data['postcode'] = 'NA'
data['city'] = 'NA'
data['state'] = 'NA'

# initialize API Nominatim
geolocator = Nominatim( user_agent='geopiRequest' )

response.raw

for i in range( len( data ) ):
    print( 'Loop : {}/{}'.format( i, len(data) ) )
    # create query
    query = str( data.loc[i,'lat'] ) + ',' + str( data.loc[i,'long'] )
    
    # API request
    response = geolocator.reverse( query, timeout=None )
    
    # populate data:
    if 'road' in response.raw['address']:
        data.loc[i, 'road'] = response.raw['address']['road']
        
    if 'house_number' in response.raw['address']:
        data.loc[i, 'house_number'] = response.raw['address']['house_number']
    
    if 'neighbourhood' in response.raw['address']:
        data.loc[i, 'neighbourhood'] = response.raw['address']['neighbourhood']
    
    if 'postcode' in response.raw['address']:
        data.loc[i, 'postcode'] = response.raw['address']['postcode']
    
    if 'city' in response.raw['address']:
        data.loc[i, 'city'] = response.raw['address']['city']
    
    if 'state' in response.raw['address']:
        data.loc[i, 'state'] = response.raw['address']['state']
    


Loop : 0/21612
Loop : 1/21612
Loop : 2/21612
Loop : 3/21612
Loop : 4/21612
Loop : 5/21612
Loop : 6/21612
Loop : 7/21612
Loop : 8/21612
Loop : 9/21612
Loop : 10/21612
Loop : 11/21612
Loop : 12/21612
Loop : 13/21612
Loop : 14/21612
Loop : 15/21612
Loop : 16/21612
Loop : 17/21612
Loop : 18/21612
Loop : 19/21612
Loop : 20/21612
Loop : 21/21612
Loop : 22/21612
Loop : 23/21612
Loop : 24/21612
Loop : 25/21612
Loop : 26/21612
Loop : 27/21612
Loop : 28/21612
Loop : 29/21612
Loop : 30/21612
Loop : 31/21612
Loop : 32/21612
Loop : 33/21612
Loop : 34/21612
Loop : 35/21612
Loop : 36/21612
Loop : 37/21612
Loop : 38/21612
Loop : 39/21612
Loop : 40/21612
Loop : 41/21612
Loop : 42/21612
Loop : 43/21612
Loop : 44/21612
Loop : 45/21612
Loop : 46/21612
Loop : 47/21612
Loop : 48/21612
Loop : 49/21612
Loop : 50/21612
Loop : 51/21612
Loop : 52/21612
Loop : 53/21612
Loop : 54/21612
Loop : 55/21612
Loop : 56/21612
Loop : 57/21612
Loop : 58/21612
Loop : 59/21612
Loop : 60/21612
Loop : 61/21612
Loop : 62/21612
Lo

In [90]:
# initialize API Nominatim
geolocator = Nominatim( user_agent='geopiRequest' )

# make query
query = '47.51, -122.26'

# API request:
response = geolocator.reverse( query )

response.raw


{'place_id': 159595021,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 236673776,
 'lat': '47.50987735',
 'lon': '-122.26001908123574',
 'display_name': '5746, South Ryan Street, Rainier Beach, Seattle, King County, Washington, 98178, United States',
 'address': {'house_number': '5746',
  'road': 'South Ryan Street',
  'neighbourhood': 'Rainier Beach',
  'city': 'Seattle',
  'county': 'King County',
  'state': 'Washington',
  'postcode': '98178',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['47.509808', '47.5099694', '-122.2600792', '-122.259959']}

In [86]:
data[['lat', 'long']].head()


,lat,long
0,47.51,-122.26
1,47.72,-122.32
2,47.74,-122.23
3,47.52,-122.39
4,47.62,-122.05


In [14]:
display( data )

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,yr_built,yr_renovated,zipcode,lat,long,year,year_week,is_waterfront
0,7129300520,2014-10-13,221900.0,3,1.00,1.0,0,3,1955,0,98178,47.5112,-122.257,2014,2014-41,no
1,6414100192,2014-12-09,538000.0,3,2.25,2.0,0,3,1951,1991,98125,47.7210,-122.319,2014,2014-49,no
2,5631500400,2015-02-25,180000.0,2,1.00,1.0,0,3,1933,0,98028,47.7379,-122.233,2015,2015-08,no
3,2487200875,2014-12-09,604000.0,4,3.00,1.0,0,5,1965,0,98136,47.5208,-122.393,2014,2014-49,no
4,1954400510,2015-02-18,510000.0,3,2.00,1.0,0,3,1987,0,98074,47.6168,-122.045,2015,2015-07,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21607,263000018,2014-05-21,360000.0,3,2.50,3.0,0,3,2009,0,98103,47.6993,-122.346,2014,2014-20,no
21608,6600060120,2015-02-23,400000.0,4,2.50,2.0,0,3,2014,0,98146,47.5107,-122.362,2015,2015-08,no
21609,1523300141,2014-06-23,402101.0,2,0.75,2.0,0,3,2009,0,98144,47.5944,-122.299,2014,2014-25,no
21610,291310100,2015-01-16,400000.0,3,2.50,2.0,0,3,2004,0,98027,47.5345,-122.069,2015,2015-02,no


In [9]:
data[['id','is_waterfront']].groupby( 'is_waterfront' ).count()

,id
is_waterfront,
no,21449
yes,163
